<a href="https://colab.research.google.com/github/grosetta/rdkit_study/blob/main/train_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#文件头代码
import sys
from google.colab import drive
drive.mount('/content/gdrive')
sys.path.insert(0,"/content/gdrive/My Drive/Colab Notebooks/python_package/")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 模块导入

In [2]:
import csv
import itertools
import operator
import numpy as np
import nltk
import os
from rdkit import Chem
from rdkit.Chem import Draw
from IPython import display
from rdkit.Chem import Descriptors
import sys
from datetime import datetime
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation,TimeDistributed
from keras.layers import LSTM,GRU
from keras.layers import Embedding
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from keras.layers import Dropout
import random
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.models import model_from_json
from keras.layers import Conv1D, MaxPooling1D

# 函数定义

In [3]:
def zinc_data_with_bracket_original():
#将文件的所有smile转存为一个列表，列表中每一个元素是一个smile
    sen_space=[]
    f = open('/content/gdrive/My Drive/data/250k_rndm_zinc_drugs_clean.smi', 'r')

    reader = csv.reader(f)
    for row in reader:
        sen_space.append(row) #此时的sen_space是列表里套列表的形式
    f.close()

    zinc_processed=[]
    for i in range(len(sen_space)):
        word1=sen_space[i]
        zinc_processed.append(word1[0])
    return zinc_processed

In [4]:
def zinc_processed_with_bracket(sen_space):
    all_smile=[]
    length=[]
    end="\n"
    element_table=["C","N","B","O","P","S","F","Cl","Br","I","(",")","=","#"]
    ring=["1","2","3","4","5","6","7","8","9","10"]

    for i in range(len(sen_space)):
        word_space=sen_space[i]
        word=[]
        j=0
        while j<len(word_space):
            word_space1=[]
            if word_space[j]=="[":
                word_space1.append(word_space[j])
                j=j+1
                while word_space[j]!="]":
                    word_space1.append(word_space[j])
                    j=j+1
                word_space1.append(word_space[j])
                word_space2=''.join(word_space1)
                word.append(word_space2)    #将smile的中括号即其内部的字母作为整体添加到word列表中
                j=j+1
            else:
                word_space1.append(word_space[j])

                if j+1<len(word_space):
                    word_space1.append(word_space[j+1])  #如果第j个字符不是最后一个，那就继续遍历
                    word_space2=''.join(word_space1)
                else:
                    word_space1.insert(0,word_space[j-1]) #如果第j个字符是最后一个
                    word_space2=''.join(word_space1)

                if word_space2 not in element_table:   #如果第j个字符不在预定的元素集合里，就照搬到word列表中 
                    word.append(word_space[j])
                    j=j+1
                else:
                    word.append(word_space2)
                    j=j+2


        word.append(end)  #给每个列表元素最后加终止符
        word.insert(0,"&") #给每个列表第一个元素前加&
        len1=len(word)
        length.append(len1) #每个元素（smile各字符隔开）长度添加到一个列表中，和all_smile中一一对应。
        all_smile.append(list(word))
    val=["\n"]
    for i in range(len(all_smile)):
        for j in range(len(all_smile[i])):   #j的取值范围各不相同，受制于各smile长度
            if all_smile[i][j] not in val:
                val.append(all_smile[i][j])

    return val, all_smile

In [5]:
def prepare_data(smiles,all_smile):
    all_smile_index=[]
    for i in range(len(all_smile)):
        smile_index=[]
        for j in range(len(all_smile[i])):
            smile_index.append(smiles.index(all_smile[i][j]))  #index() 函数用于从列表中找出某个值第一个匹配项的索引位置
        all_smile_index.append(smile_index)
    X_train=all_smile_index         #X_train表示smile中符号在valcabulary中对应的序号
    y_train=[]
    for i in range(len(X_train)):

        x1=X_train[i]    #smile转为由valcabulary中对应序号组成的列表
        x2=x1[1:len(x1)]    
        x2.append(0)
        y_train.append(x2)

    return X_train,y_train #y_train和X_train错开一位

In [ ]:
#定义训练模型的保存函数
def save_model(model):
    # serialize model to JSON
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to disk")

# 代码主体

In [6]:
smile=zinc_data_with_bracket_original()
valcabulary,all_smile=zinc_processed_with_bracket(smile)
print(valcabulary)
print(len(all_smile))

['\n', '&', 'C', '(', ')', 'c', '1', '2', 'o', '=', 'O', 'N', '3', 'F', '[C@@H]', 'n', '-', '#', 'S', 'Cl', '[O-]', '[C@H]', '[NH+]', '[C@]', 's', 'Br', '/', '[nH]', '[NH3+]', '4', '[NH2+]', '[C@@]', '[N+]', '[nH+]', '\\', '[S@]', '5', '[N-]', '[n+]', '[S@@]', '[S-]', '6', '7', 'I', '[n-]', 'P', '[OH+]', '[NH-]', '[P@@H]', '[P@@]', '[PH2]', '[P@]', '[P+]', '[S+]', '[o+]', '[CH2-]', '[CH-]', '[SH+]', '[O+]', '[s+]', '[PH+]', '[PH]', '8', '[S@@+]']
249456


In [7]:
 X_train,y_train=prepare_data(valcabulary,all_smile)

In [8]:
#maxlen：None或整数，为序列的最大长度。大于此长度的将被截断，小于此长度的序列将在后面填0.
#padding：pre或post，确定当需要补0时，在序列的起始还是结尾补。
#truncating：pre或post，确定需要截断序列时，从起始还是结尾截断。
#value：浮点数，用于填充序列
X= pad_sequences(X_train, maxlen=81, dtype='int32', padding='post', truncating='pre', value=0.)
y = pad_sequences(y_train, maxlen=81, dtype='int32', padding='post', truncating='pre', value=0.)

In [9]:
y_train_one_hot = np.array([to_categorical(sent_label, num_classes=len(valcabulary)) for sent_label in y])
print (y_train_one_hot.shape)

vocab_size=len(valcabulary)
embed_size=len(valcabulary)

(249456, 81, 64)


In [ ]:
N=X.shape[1]
model = Sequential()

model.add(Embedding(input_dim=vocab_size, output_dim=len(valcabulary), input_length=N,mask_zero=False))
model.add(GRU(256, input_shape=(81,64),activation='tanh',return_sequences=True))
#model.add(LSTM(output_dim=256, input_shape=(81,64),activation='tanh',return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(256,activation='tanh',return_sequences=True))
#model.add(LSTM(output_dim=1000, activation='sigmoid',return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(embed_size, activation='softmax')))
optimizer=Adam(lr=0.01)
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.fit(X,y_train_one_hot,epochs=100, batch_size=512,validation_split=0.1)

In [ ]:
save_model(model)